# This Notebook is made to conduct a T-Test to show if different Scheduling have a statistical significance on the nDCG and or on the rND

In [1]:
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.power as smp
from resultsHelper import cohen_d, perform_shapiro_test

In [7]:
df = pd.read_csv('../Results/DirectRankerAdv/Adult/T-Test/results.csv')
# Lists containing the performance metric for each schedule
print(df['Schedule'].unique())

['[1; 1]' '[3; 5]']


In [9]:
a = '[1; 1]'
b = '[3; 5]'
schedule_a = list(df[df['Schedule']==a]['nDCG'])
schedule_b = list(df[df['Schedule']==b]['nDCG'])

### To check if the data is normal the Shapiro Wilk test will be performed

In [10]:
# Perform the test for 'nDCG'
perform_shapiro_test(df, 'nDCG')

# Perform the test for 'rND'
perform_shapiro_test(df, 'rND')


Shapiro-Wilk Test for nDCG in Schedule [1; 1]: Statistic=0.9745062589645386, p-value=0.668184757232666
	Null hypothesis not rejected - data is normal
Shapiro-Wilk Test for nDCG in Schedule [3; 5]: Statistic=0.9329608082771301, p-value=0.058879170566797256
	Null hypothesis not rejected - data is normal
Shapiro-Wilk Test for rND in Schedule [1; 1]: Statistic=0.9645545482635498, p-value=0.4026342034339905
	Null hypothesis not rejected - data is normal
Shapiro-Wilk Test for rND in Schedule [3; 5]: Statistic=0.979521632194519, p-value=0.8126932978630066
	Null hypothesis not rejected - data is normal


### Now calculate Cohen's d to validate that the sample size is engouh

In [11]:
# Calculate Cohen's d
d = cohen_d(schedule_a, schedule_b)
print("Cohen's d:", d)

Cohen's d: 0.12407553938479315


In [13]:
effect_size = 0.124
alpha = 0.05       
power = 0.8        
ratio = 1          

# Calculate the sample size
sample_size = smp.TTestIndPower().solve_power(effect_size=effect_size,
                                              power=power,
                                              alpha=alpha,
                                              ratio=ratio,
                                              alternative='two-sided')

print(f"Required sample size: {sample_size:.2f} per group")

Required sample size: 1021.88 per group


In [6]:
schedule_a = list(df[df['Schedule']==a]['rND'])
schedule_b = list(df[df['Schedule']==b]['rND'])
# Perform the t-test
t_stat, p_value = stats.ttest_ind(schedule_a, schedule_b)

# Check significance
alpha = 0.05
print(p_value)
if p_value < alpha:
    print("Reject the null hypothesis - suggest the means are different")
else:
    print("Do not reject the null hypothesis - no suggestion that the means are different")

7.695420725019729e-11
Reject the null hypothesis - suggest the means are different
